# Yelp Restaurant Rating Prediction

## Project Description
The project goal is to predict restaurant overall ratings on yelp
in New York City, using multiple features of restaurants that we will
extract using the yelp API. Our motivation is to help determine how
successful a new restaurant business might be, given certain known
characteristics of it.

We will use the Yelp API, with the help of pandas, to acquire raw data
from restaurants. Then we will extract reasonable features such as
location, open hours, whether it takes reservations, whether it has
delivery service, whether there is parking space, and whether it
provides free wifi etc., from the parsed data, and combine with the
overall ratings, which is a numerical value ranging from 0 to 5, as
labels.

We will model the rating distribution over the different features that
we extract and create, and analyse how much each feature shifts our
distribution. Using our results from this we will select good features
to train on machine learning models.

Using the labeled features that we construct, we will train different
machine learning models like linear regression, nonlinear regression,
logistic regression as well as neural networks, then make some
predictions, and compare the accuracy obtained from them.

## Team Members
Jun Hee Kim, Nikhil Rangarajan, Sander Shi

## Procedure
* [Data Gathering from API](#step-1)
* [Feature Extraction with Parsing](#step-2)
* [Feature Analysis and Variable Selection](#step-3)
* [Setup of Models](#step-4)
* [Cross Validation](#step-5)
* [Final Analysis](#step-6)

## Part 0: Imports and Definitions of Constants

We will be using `pandas` to parse the data and `tensorflow` to construct the machine learning models. We will also be using the Yelp API to gather the data. In order to
use the Yelp API, we need to create an API key to use in our API requests.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import requests
import json
import time
import pickle
import matplotlib.pyplot as plt
from collections import Counter
import scipy.stats as stats

API_URL = "https://api.yelp.com/v3/businesses"
SEARCH_URL = API_URL + "/search"
RESTAURANT_IDS = "rid.pkl"

with open("./API_KEY", 'r') as f:
    api_key = f.readline().strip()
API_HEADERS = {
    'Authorization': ' '.join(['Bearer', api_key])
}

<a id="step-1"></a>

## Part 1: Data Gathering from API

In this step we will use the Yelp API to gather restaurant pages, then extract
information using business search API requests.

Since the Yelp API will only allow us to load business information from up to 1000
distinct businesses on each search, we will first get the categories of the
restaurants, then perform a search query for each category.

In [2]:
def get_restaurant_categories(url):
    """
    This function gets all restaurant categories.
    
    @input url: URL to json file containing restaurant categories.
    @type url: String.
    
    @return: List of restaurant categories.
    @rtype: List of String
    """
    cats = json.load(open(url))
    return [cat['alias'] for cat in cats if 'restaurants' in cat['parents']]

categories = get_restaurant_categories('categories.json')

Now that we have the restaurant categories, we can write a function to find all
restaurant IDs, and create a Pandas DataFrame. To aid debugging, we will dump
the dataframe into a pickle file so that we will not have to regenerate each time.

In [3]:
def get_req_json(url, params=None):
    """
    This function is a wrapper for a get request to the API.
    
    @input url: The API url.
    @type url: String.
    @input params: The parameters to pass to the get request.
    @type params: Dict.
    
    @return: A json response.
    @rtype: JSON
    """
    response = requests.get(url=url, headers=API_HEADERS, params=params)
    return response.json()


def find_restaurants(url, categories):
    """
    This function loads all restaurant data from restaurants in NYC.
    
    @input url: The API url.
    @type url: String.
    @input categories: The restaurant categories.
    @type categories: List of String.
    
    @return: A Pandas DataFrame containing the restaurant IDs.
    @rtype: pandas.DataFrame.
    """
    restaurants = []
    for cat in categories:
        params = {
            'term': 'restaurants',
            'location': 'NYC',
            'categories': cat
        }
        content = get_req_json(url, params)
        total = min(1000, content['total'])
        i = 0
        while i < total:
            limit = min(50, total - i)
            params['limit'] = limit
            params['offset'] = i
            content = get_req_json(url, params)
            restaurants.extend([b['id'] for b in content['businesses']])
            i += limit
    return pd.DataFrame({'id': list(set(restaurants))})

# Uncomment the following lines to get the restaurant IDs and recreate the pickle file.
# restaurants = find_restaurants(SEARCH_URL, categories)
# pickle.dump(restaurants, open(RESTAURANT_IDS, "wb"))

# Load restaurant IDs from pickle file
restaurant_ids = pickle.load(open(RESTAURANT_IDS, 'rb'))
restaurant_ids.head()

,id
0,UTNSUIY2QW6RYNa0xLa9gg
1,h5WkewI6U7NjLB_n6WgdvQ
2,yOPbrFZ2ZUsyQWsedP6gUg
3,2ngodDyMj6M0szyBbGSJ5g
4,8ghGdEWOkGWUNeGrtdiaDQ


<a id="step-2"></a>

## Part 2: Feature Extraction with Parsing

Now that we have all the restaurant ids, in this step we can do a business lookup
to get the specific data of the restaurants.

In [4]:
def get_data(rids, api_url):
    """
    This function uses the Yelp business lookup API to get all data from each
    restaurant.
    
    @input api_url: The API url.
    @type api_url: String.
    @input rids: The dataframe containing all restaurant IDs.
    @type rids: pandas.DataFrame.
    
    @return: A list of restaurant information.
    @rtype: List of Json Object.
    """
    info = []
    for idx, rid in rids.iterrows():
        url = "/".join([api_url, rid['id']])
        content = get_req_json(url)
        info.append(content)
    return info

# Uncomment the following two lines to get the data from the API and save to pickle.
# all_data = get_data(restaurant_ids, API_URL)
# pickle.dump(all_data, open("restaurant_data.pkl", "wb"))

# Load all restaurant data from pickle file.
all_data = pickle.load(open("restaurant_data.pkl", 'rb'))
all_data[0]

{'id': 'UTNSUIY2QW6RYNa0xLa9gg',
 'alias': 'new-pizza-professor-brooklyn-brooklyn',
 'name': 'New Pizza Professor - Brooklyn',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/zpkQadE2UF3DjvOBUHF9PA/o.jpg',
 'is_claimed': True,
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/new-pizza-professor-brooklyn-brooklyn?adjust_creative=ZAj1bx8bJHOcikvnMXVxEg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=ZAj1bx8bJHOcikvnMXVxEg',
 'phone': '+13475874277',
 'display_phone': '(347) 587-4277',
 'review_count': 16,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'kosher', 'title': 'Kosher'}],
 'rating': 2.5,
 'location': {'address1': '1824 Avenue M',
  'address2': '',
  'address3': None,
  'city': 'Brooklyn',
  'zip_code': '11230',
  'country': 'US',
  'state': 'NY',
  'display_address': ['1824 Avenue M', 'Brooklyn, NY 11230'],
  'cross_streets': '19th St & Bay Ave'},
 'coordinates': {'latitude': 40.6184002, 'longitude': -73.9564177},
 'photos':

Using the data that we obtained from the API, we can extract useful information from it to build
a dataframe of features.

In [5]:
def extract_features(data):
    """
    This function creates a dataframe of features given a list of restaurant data.
    
    @input data: A list of restaurant data obtained from the Yelp API.
    @type api_url: List of JSON Object.
    
    @return: A Pandas DataFrame containing the restaurant features.
    @rtype: pandas.DataFrame.
    """
    prices, ratings, pickups, deliveries= [], [], [], []
    reservations, is_claimed, zipcodes, categories = [], [], [], []
    days = [2, 4, 5, 6]
    hours = dict()
    for day in days:
        hours[day] = []
    for content in (x for x in data if 'error' not in x):
        prices.append(len(content.get('price', "")))
        ratings.append(float(content.get('rating', 0)))
        transactions = content.get('transactions', [])
        pickups.append(int("pickup" in transactions))
        deliveries.append(int("delivery" in transactions))
        reservations.append(int("restaurant_reservation" in transactions))
        is_claimed.append(int(content.get('is_claimed', False)))
        if ('location' not in content) or ('zip_code' not in content['location']):
            zipcodes.append('0')
        else:
            zipcodes.append(content['location']['zip_code'])
        for day in days:
            if "hours" in content:
                open_hours = content["hours"][0]["open"]
                stats = [h for h in open_hours if h['day'] == day]
                is_overnight = int(True in [h['is_overnight'] for h in stats])
                start = min([h['start'] for h in stats], default='0000')
                end = max([h['end'] if h['end'] != '0000' else '2400' for h in stats], default='0000')
                hours[day].append([is_overnight, start, end])
            else:
                hours[day].append([0, '1200', '1800'])
        cat_list = [c['alias'] for c in content['categories']]
        categories.append(np.random.choice(cat_list, 1)[0])
        
    for day in days:
        hours[day] = np.array(hours[day])
    df_dict = {
        'price': prices,
        'category': categories,
        'pickup': pickups,
        'delivery': deliveries,
        'restaurant_reservation': reservations,
        'is_claimed': is_claimed,
        'zip_code': zipcodes,
        'rating': ratings,
        'wed_is_overnight': hours[2][:, 0].astype(int),
        'fri_is_overnight': hours[4][:, 0].astype(int),
        'sat_is_overnight': hours[5][:, 0].astype(int),
        'sun_is_overnight': hours[6][:, 0].astype(int),
        'wed_start': hours[2][:, 1],
        'fri_start': hours[4][:, 1],
        'sat_start': hours[5][:, 1],
        'sun_start': hours[6][:, 1],
        'wed_end': hours[2][:, 2],
        'fri_end': hours[4][:, 2],
        'sat_end': hours[5][:, 2],
        'sun_end': hours[6][:, 2]
    }
    return pd.DataFrame(df_dict)

# Extract features from the loaded restaurant data
labeled_features = extract_features(all_data)
labeled_features.head()

,category,delivery,fri_end,fri_is_overnight,fri_start,is_claimed,pickup,price,rating,restaurant_reservation,sat_end,sat_is_overnight,sat_start,sun_end,sun_is_overnight,sun_start,wed_end,wed_is_overnight,wed_start,zip_code
0,kosher,1,1445,0,1100,1,1,0,2.5,0,0300,1,2030,0200,1,1100,0200,1,1100,11230
1,comfortfood,1,0400,1,1600,1,1,2,4.0,1,0400,1,1100,2400,0,1100,0200,1,1600,10016
2,burgers,1,0300,1,1200,1,1,2,3.5,0,0300,1,1100,0200,1,1100,0200,1,1200,10025
3,greek,1,2200,0,1200,0,1,1,3.0,0,2200,0,1200,2200,0,1200,2200,0,1200,11105
4,creperies,1,2000,0,0800,1,1,2,4.5,0,2000,0,0900,1800,0,0900,1900,0,0800,11231


<a id="step-3"></a>

## Part 3: Feature Analysis and Variable Selection

Now that we have the features, we can separately analyse them and see how each feature
contributes to the rating.

### 1) Price
There are four levels (1, 2, 3, or 4) in price. Let's plot the frequency distribution of ratings for each price level to see if they differ alot, and then run an one-way ANOVA test to see if the population means of the ratings are different for at least two price levels.

In [6]:
plt.rcParams["figure.figsize"] = [12,8]

def barplot_each_price(price_levels, rating, df):
    """
    Input: 
    - price_levels is the list of possible price levels
    - rating is the list of possible ratings
    - df is the pandas dataframe containing the data
    
    This function plots the bar chart of the ratings for each of the price levels
    """
    for i in price_levels:
        price = i
        # ratings with this price level
        this_price_df = df.loc[df['price']==price]
        data = this_price_df['rating']
        counter = Counter(data)
        freq = [counter[rate] for rate in rating]
        plt.subplot(2,2,i)
        plt.subplots_adjust(hspace=0.4, wspace=0.4)
        plt.bar(rating, freq, width=0.2)
        plt.xlabel("Rating")
        plt.ylabel("Frequency")
        plt.ylim(ymin=0, ymax=3000)
        plt.xticks(rating, rating)
        plt.title("Price Level: %s"%price)

barplot_each_price(price_levels=[1,2,3,4], rating=np.linspace(1, 5, 9), df=labeled_features)

We've set the y-axis range to be the same for all four subplots so that visually comparing becomes easier. The shapes of the rating frequencies appear very different. But such a visual impression might be due to the number of restaurants being so different for each price level rather than the distributions being different. So let's perform ANOVA to yield some formal conclusion.

In [7]:
def price_oneway_ANOVA(price_levels, df):
    """
    Input: 
    - price_levels is the list of possible price levels
    - df is the pandas dataframe containing the data
    
    This function returns the output of one-way ANOVA on ratings across different price levels
    """
    # This dictionary will have each key being each price level, and the corresponding value will be the array
    # of the ratings of the restaurants with that price level
    price_rating_dict = dict()
    for price in price_levels:
        this_price_df = df.loc[df['price']==price]
        price_rating_dict[price] = np.array(this_price_df['rating'])
    return stats.f_oneway(*price_rating_dict.values())

price_oneway_ANOVA(price_levels=[1,2,3,4], df=labeled_features)

F_onewayResult(statistic=48.77633760216919, pvalue=2.367994922849704e-31)

According to the ANOVA output, at $\alpha=0.01$, the p-value is less than $\alpha$, so we conclude at least two price levels have population rating means that are different. A limitation of one-way ANOVA is that we can't conclude which groups have different population rating means, but in our context, knowing that the rating means differ in at least two levels lets us conclude price is an important feature.

### 2) Transactions (Pickup, Delivery, Reservation)

We've parsed the transaction features as: "pickup" is 1 if pickup is available in that restaurant and 0 otherwise, and similarly for "delivery" and "restaurant_reservation". For each of the three transaction features, let's use two-sample t-test (one group of the restaurants that have that transaction, and the other group of those that don't) to determine if the two groups have differnt population rating means.

In [8]:
def two_sample_ttest(feature_name, values, df):
    """
    Input: 
    - feature_name is the name of the feature
    - values is the list of 2 possible values (which determine the 2 groups)
    - df is the pandas dataframe containing the data
    
    This function returns the output of one-way ANOVA on ratings across different price levels
    """
    feature_rating_dict = dict()
    for value in values:
        this_value_df = df.loc[df[feature_name]==value]
        feature_rating_dict[value] = np.array(this_value_df['rating'])
    return stats.ttest_ind(*feature_rating_dict.values(), equal_var=False)

transaction_vars = ['pickup', 'delivery', 'restaurant_reservation']
for feature in transaction_vars:
    print(feature, ":", two_sample_ttest(feature_name=feature, values=[0,1], df=labeled_features))

pickup : Ttest_indResult(statistic=3.448779997551905, pvalue=0.0005645728857015097)
delivery : Ttest_indResult(statistic=5.562932502127382, pvalue=2.6951061936021385e-08)
restaurant_reservation : Ttest_indResult(statistic=-14.809667384237882, pvalue=1.2817988406449425e-47)


According to the t-test output, for each of the three transaction features, at $\alpha=0.01$, the p-value is less than $\alpha$, so we conclude the two groups have population rating means that are different. One might think that all three features should be included in the model.

But common sense makes us suspicious about correlations among the transaction features, and in general, correlated features can cause problems in modeling (e.g. multicollinearity problem in linear regression). Let's examine the correlation coefficient for each pair among the three.

In [9]:
def print_pcc(list_vars, df):
    """
    Input
    - list_vars is the list of feature names in which the Pearson Correlation Coefficient for each pair is computed
    - df is the pandas dataframe containing the data
    
    This function prints the Pearson Correlation Coefficient for each pair of the features provided in the list
    """
    for i in range(0, len(list_vars)):
        for j in range(i+1, len(list_vars)):
            corr = np.corrcoef(df[list_vars[i]], df[list_vars[j]])[0,1]
            print(list_vars[i],  ",", list_vars[j], corr)

print_pcc(transaction_vars, df=labeled_features)

pickup , delivery 0.9303345646034946
pickup , restaurant_reservation 0.11267822332819287
delivery , restaurant_reservation 0.09576221912941747


Pickup and delivery are highly correlated, but the other two pairs are fine. So among the three, we decide to include reservation and only one of pickup and delivery. Between the two correlated features, we choose delivery since its t-test p-value is lower than pickup's.

### 3) Zip Code

Let's plot a bubble plot of the ratings and zip codes.

In [10]:
plt.rcParams["figure.figsize"] = [16,10]

def bubbleplot_two_vars(var1, var2, df):
    """
    Input
    - var1 is the name of the first variable (which is a column in the dataframe)
    - var1 is the name of the second variable (which is a column in the dataframe)
    - df is the dataframe containing the data
    
    This function plots a bubble plot of the two variables in the dataframe
    """
    pairs = np.array(df[[var1, var2]]).astype(str)
    label_x, x = np.unique(pairs[:,0], return_inverse=True)
    label_y, y = np.unique(pairs[:,1], return_inverse=True)
    xy, cnts = np.unique((x,y), axis=1, return_counts=True)
    plt.scatter(xy[0], xy[1], s=cnts*8)
    plt.xlabel("Zip Code")
    plt.ylabel("Rating")
    plt.title("Rating vs Zip Code; Point size indicates count")
    plt.xticks(range(len(label_x)), label_x)
    plt.yticks(range(len(label_y)), label_y)

bubbleplot_two_vars(var1='zip_code', var2='rating', df=labeled_features)

We can see that in some zip codes, the ratings of 4.0 and 3.5 are clearly dominant, while for other zip codes, all the ratings are similarly frequent. This plot suggests that ratings do differ for different zip codes.

Now, let's perform one-way ANOVA on the zip codes to check if at least two zip codes have different population mean of ratings. Few rows in our dataframe have missing zip code information and have them recorded as either "0", "", or None. So we exclude them when performing ANOVA.

In [11]:
def zipcode_oneway_ANOVA(zipcodes, df):
    """
    Input: 
    - zipcodes is the list of zip codes
    - df is the pandas dataframe containing the data
    
    This function returns the output of one-way ANOVA on ratings across different zip codes
    """
    # This dictionary will have each key being each zip code, and the corresponding value will be the array
    # of the ratings of the restaurants with that zip code
    zipcode_rating_dict = dict()
    for zipcode in zipcodes:
        this_zipcode_df = df.loc[df['zip_code']==zipcode]
        zipcode_rating_dict[zipcode] = np.array(this_zipcode_df['rating'])
    return stats.f_oneway(*zipcode_rating_dict.values())

unique_zipcodes = labeled_features['zip_code'].unique()
unique_zipcodes = unique_zipcodes[unique_zipcodes != "0"]
unique_zipcodes = unique_zipcodes[unique_zipcodes != ""]
unique_zipcodes = unique_zipcodes[unique_zipcodes != None]
zipcode_oneway_ANOVA(zipcodes=unique_zipcodes, df=labeled_features)

F_onewayResult(statistic=4.428869810981315, pvalue=3.9649519708657156e-88)

According to the ANOVA output, at $\alpha=0.01$, the p-value is less than $\alpha$, so we conclude at least two zip codes have population rating means of restaurants that are different, and so we consider zip code an important feature. 

**NOTE**: Since we'll include it in our modeling, we should remove the few rows with invalid zip codes ("0", "", or None) from our dataframe before setting up models.

### 4) Claimed

A claimed business page in Yelp is one that has been claimed by the owner or representative of the business through our verification process. Let's plot the frequency of ratings for the claimed restaurants and that for nonclaimed restaurants, and then perform two-sample t-test.

In [12]:
plt.rcParams["figure.figsize"] = [10,4]

def barplot_claimed(rating, df):
    """
    Input: 
    - rating is the list of possible ratings
    - df is the pandas dataframe containing the data
    
    This function overplots the bar chart of the ratings for restaurants claimed and those that are not.
    """
    for i in [0,1]:
        subplot_index = i + 1
        # ratings of claimed restaurants
        this_price_df = df.loc[df['is_claimed']==i]
        data = this_price_df['rating']
        counter = Counter(data)
        freq = [counter[rate] for rate in rating]
        plt.subplot(1,2,subplot_index)
        plt.subplots_adjust(hspace=0.4, wspace=0.4)
        plt.bar(rating, freq, width=0.2)
        plt.xlabel("Rating")
        plt.ylabel("Frequency")
        plt.ylim(ymin=0, ymax=4000)
        plt.xticks(rating, rating)
        plt.title("Claimed" if i==1 else "Not Claimed")

barplot_claimed(rating=np.linspace(1, 5, 9), df=labeled_features)

The bar charts suggest that the frequencies are considerably different, especially in terms of how dominantly frequent ratings of 3.5 or 4 are. Let's perform two-sample t-test.

In [13]:
two_sample_ttest(feature_name='is_claimed', values=[0,1], df=labeled_features)

Ttest_indResult(statistic=-19.620624050804704, pvalue=2.4638528098179564e-83)

According to the t-test output, at $\alpha=0.01$, the p-value is less than $\alpha$, so we conclude the two groups have population rating means that are different and thus treat whether or not the restaurant is claimed as a meaningful feature.

<a id="step-4"></a>

## Part 4: Setup of Models

We need to construct a couple machine learning models for us to train to see which
one gives us the best testing accuracy.

In [14]:
class ANN(object):
    """
    This is a neural network.
    """
    def __init__(self, layers=[5, 5]):
        self.layers = layers
        
class LogisticRegression(object):
    """
    This is the logistic regression model.
    """
    def __init__(self):
        self.theta = np.zeros(100)

<a id="step-5"></a>

## Part 5: Cross Validation

Now we will use cross-validation to determine which model produces the highest
validation accuracy for our dataset, then pick this model.

In [15]:
def pick_model():
    pass

<a id="step-6"></a>

## Part 6: Final Analysis

Our results were amazing, so yeah.